In [4]:
from pathlib import Path
from gnpy.tools.json_io import load_equipment, load_network
from gnpy.core.utils import lin2db, automatic_nch
from gnpy.core.elements import Transceiver, Roadm, Fiber, Edfa
from gnpy.tools.worker_utils import designed_network, transmission_simulation
import networkx as nx
import sys
import traceback
import numpy as np

In [8]:
# === 1) PARÁMETROS DEL CANAL ÓPTICO (editables desde el código) ===
f_min = 191.3e12             # Hz
f_max = 196.1e12             # Hz
spacing = 50e9               # Hz
baud_rate = 31.57e9          # Hz
roll_off = 0.15              # sin unidad
tx_osnr = 35                 # dB
sys_margins = 2              # dB
power_dbm = 2               # dBm
power_range_db = [0, 0, 1]   # [inicio, fin, paso] en dB

# === Archivos ===
EQPT = Path("eqpt_configv1.json")
TOPO = Path("topologiaEC.json")

def tipo(n):
    if isinstance(n, Transceiver): return "TX/RX"
    if isinstance(n, Roadm): return "Roadm"
    if isinstance(n, Fiber): return "Fiber"
    if isinstance(n, Edfa): return "EDFA"
    return "Otro"

# === Entrada del usuario ===
try:
    num_rutas = int(input("Número de rutas a calcular: ").strip())
    if num_rutas < 1:
        raise ValueError
except ValueError:
    sys.exit("Entradas inválidas. Debes ingresar un número válido.")

# === Cargar red y modificar parámetros del canal óptico ===
equipment = load_equipment(EQPT)
network = load_network(TOPO, equipment)

# Sobrescribir los parámetros del canal óptico
si = list(equipment["SI"].values())[0]
si.f_min = f_min
si.f_max = f_max
si.spacing = spacing
si.baud_rate = baud_rate
si.roll_off = roll_off
si.tx_osnr = tx_osnr
si.sys_margins = sys_margins
si.power_dbm = power_dbm
si.power_range_db = power_range_db

# Mostrar parámetros en consola
print("\n# 1) PARÁMETROS DEL CANAL ÓPTICO")
print(f"  f_min:           {f_min/1e12:.2f} THz")
print(f"  f_max:           {f_max/1e12:.2f} THz")
print(f"  spacing:         {spacing/1e9:.2f} GHz")
print(f"  baud_rate:       {baud_rate/1e9:.2f} Gbaud")
print(f"  roll_off:        {roll_off}")
print(f"  tx_osnr:         {tx_osnr} dB")
print(f"  sys_margins:     {sys_margins} dB")
print(f"  power_dbm:       {power_dbm} dBm")
print(f"  power_range_db:  {power_range_db}")

# Cálculo manual del número de canales
nch = int(np.floor((f_max - f_min) / spacing)) + 1
print(f"  Número de canales: {nch}")

uid2node = {n.uid: n for n in network.nodes()}

# Transceivers disponibles
tx_nodes = [n for n in network.nodes() if isinstance(n, Transceiver)]
tx_uids = [n.uid for n in tx_nodes]

print("\nTransceivers disponibles:")
for i, uid in enumerate(tx_uids, 1):
    print(f"  {i}. {uid}")

src_uid = input("\nIngrese el UID del transceiver origen (TX): ").strip()
if src_uid not in tx_uids:
    sys.exit(f"Transceiver '{src_uid}' no encontrado.")

dst_uids = [uid for uid in tx_uids if uid != src_uid]
print("\nTransceivers destino disponibles:")
for i, uid in enumerate(dst_uids, 1):
    print(f"  {i}. {uid}")

dst_uid = input("\nIngrese el UID del transceiver destino (RX): ").strip()
if dst_uid not in dst_uids:
    sys.exit(f"Transceiver '{dst_uid}' no válido o igual al origen.")

src, dst = uid2node[src_uid], uid2node[dst_uid]

# Crear grafo para rutas
G = nx.DiGraph()
for u, v in network.edges():
    G.add_edge(u.uid, v.uid)

print(f"\nBuscando hasta {num_rutas} rutas entre {src.uid} → {dst.uid}...\n")

# Buscar rutas simples
paths_uid = list(nx.shortest_simple_paths(G, src_uid, dst_uid))[:num_rutas]

# Evaluar rutas
resultados = []
for i, uid_path in enumerate(paths_uid, 1):
    try:
        path_nodes = [uid2node[uid] for uid in uid_path]
        net_designed, req, ref_req = designed_network(
            equipment, network,
            source=src.uid,
            destination=dst.uid,
            nodes_list=[n.uid for n in path_nodes],
            loose_list=['STRICT'],
            args_power=power_dbm
        )
        path, _, _, infos = transmission_simulation(equipment, net_designed, req, ref_req)
        receiver = path[-1]

        # Copiar métricas desde infos hacia el receptor
        if hasattr(infos, 'snr'):
            receiver.snr = infos.snr
        if hasattr(infos, 'osnr_ase'):
            receiver.osnr_ase = infos.osnr_ase

        if hasattr(receiver, 'snr_01nm') and hasattr(receiver, 'osnr_ase_01nm'):
            dist_total = sum(
                getattr(n, 'length', getattr(getattr(n, 'params', None), 'length', 0))
                for n in path if isinstance(n, Fiber)
            )
            resultados.append({
                'ruta': path,
                'receiver': receiver,
                'gsnr_01nm': receiver.snr_01nm.mean(),
                'gsnr_bw': receiver.snr.mean() if hasattr(receiver, 'snr') else None,
                'osnr_01nm': receiver.osnr_ase_01nm.mean(),
                'osnr_bw': receiver.osnr_ase.mean() if hasattr(receiver, 'osnr_ase') else None,
                'dist': dist_total
            })
        else:
            print(f"⚠️ Ruta {i} falló: receptor sin métricas 0.1nm.")
    except Exception as e:
        print(f"⚠️ Ruta {i} falló: {e}")
        traceback.print_exc()

# ==== 2) Selección de criterio y ordenación ====
criterio = input("¿Optimizar por (g)snr o (d)istancia? ").strip().lower()
if criterio.startswith('d'):
    resultados.sort(key=lambda r: (r['dist'], -r['gsnr_01nm']))
    print("→ Ordenando rutas por distancia (ascendente).")
else:
    resultados.sort(key=lambda r: (-r['gsnr_01nm'], r['dist']))
    print("→ Ordenando rutas por GSNR (descendente).")

# ==== 3) Mostrar rutas encontradas ====
if not resultados:
    print("No se encontró ninguna ruta válida.")
else:
    for idx, res in enumerate(resultados[:num_rutas], 1):
        ruta = res['ruta']
        receiver = res['receiver']
        print(f"\nRuta {idx} (Criterio: {'Distancia' if criterio.startswith('d') else 'GSNR'}):")
        print("Idx | {:>30} | {:<10}".format("UID", "Tipo"))
        print("-" * 50)
        for i, n in enumerate(ruta):
            print(f"{i:3d} | {n.uid:>30} | {tipo(n):<10}")
        print("\nDetalles de fibras:")
        for node in ruta:
            if isinstance(node, Fiber):
                length = getattr(node, 'length', getattr(node.params, 'length', 0))
                print(f"  {node.uid}: {length/1000:.2f} km")
        print(f"\nReceptor final: {receiver.uid}")
        print(f"  GSNR (0.1nm):    {res['gsnr_01nm']:.2f} dB")
        print(f"  GSNR (signal):   {res['gsnr_bw']:.2f} dB" if res['gsnr_bw'] else "  GSNR (signal):   N/A")
        print(f"  OSNR (0.1nm):    {res['osnr_01nm']:.2f} dB")
        print(f"  OSNR (signal):   {res['osnr_bw']:.2f} dB" if res['osnr_bw'] else "  OSNR (signal):   N/A")
        print(f"\n  Distancia total recorrida:       {res['dist']/1000:.2f} km")


	WARNING missing type_variety attribute in eqpt_config.json[Roadm]
	default value is type_variety = default




# 1) PARÁMETROS DEL CANAL ÓPTICO
  f_min:           191.30 THz
  f_max:           196.10 THz
  spacing:         50.00 GHz
  baud_rate:       31.57 Gbaud
  roll_off:        0.15
  tx_osnr:         35 dB
  sys_margins:     2 dB
  power_dbm:       2 dBm
  power_range_db:  [0, 0, 1]
  Número de canales: 97

Transceivers disponibles:
  1. trx_1
  2. trx_2
  3. trx_3
  4. trx_4
  5. trx_5
  6. trx_6
  7. trx_11
  8. trx_12
  9. trx_10
  10. trx_7
  11. trx_8
  12. trx_9

Transceivers destino disponibles:
  1. trx_1
  2. trx_2
  3. trx_3
  4. trx_4
  5. trx_5
  6. trx_6
  7. trx_11
  8. trx_12
  9. trx_10
  10. trx_8
  11. trx_9

Buscando hasta 3 rutas entre trx_7 → trx_11...

→ Ordenando rutas por GSNR (descendente).

Ruta 1 (Criterio: GSNR):
Idx |                            UID | Tipo      
--------------------------------------------------
  0 |                          trx_7 | TX/RX     
  1 |                        roadm_7 | Roadm     
  2 | Edfa_booster_roadm_7_to_fiber (7 → Edfa11_7_1